In [3]:
# Cell 1: The Triage Safety Net (Clinical Severity Mapping)
# 🔴 RED = Critical Emergency (Go to ER immediately)
# 🟡 YELLOW = Urgent / Warning (Schedule a doctor's appointment soon)
# 🟢 GREEN = Minor / Non-Urgent (Home care, rest, OTC management)

TRIAGE_SEVERITY = {
    # --- RED FLAG (CRITICAL EMERGENCY) ---
    'Anaphylaxis': 'RED',
    'Acute pulmonary edema': 'RED',
    'Boerhaave': 'RED', # Esophageal rupture
    'Ebola': 'RED',
    'Epiglottitis': 'RED',
    'Possible NSTEMI / STEMI': 'RED', # Heart attack
    'Pulmonary embolism': 'RED',
    'Spontaneous pneumothorax': 'RED', # Collapsed lung
    'Unstable angina': 'RED',
    'Larygospasm': 'RED',
    'Myocarditis': 'RED',
    'Guillain-Barré syndrome': 'RED', # Can halt breathing
    'PSVT': 'RED', # Dangerous heart arrhythmia
    'Bronchospasm / acute asthma exacerbation': 'RED', 

    # --- YELLOW FLAG (URGENT / DOCTOR VISIT) ---
    'Acute COPD exacerbation / infection': 'YELLOW',
    'Acute dystonic reactions': 'YELLOW',
    'Atrial fibrillation': 'YELLOW',
    'Bronchiectasis': 'YELLOW',
    'Bronchiolitis': 'YELLOW',
    'Chagas': 'YELLOW',
    'Croup': 'YELLOW',
    'HIV (initial infection)': 'YELLOW',
    'Inguinal hernia': 'YELLOW',
    'Myasthenia gravis': 'YELLOW',
    'Pancreatic neoplasm': 'YELLOW',
    'Pericarditis': 'YELLOW',
    'Pneumonia': 'YELLOW',
    'Pulmonary neoplasm': 'YELLOW',
    'SLE': 'YELLOW', # Lupus
    'Sarcoidosis': 'YELLOW',
    'Spontaneous rib fracture': 'YELLOW',
    'Stable angina': 'YELLOW',
    'Tuberculosis': 'YELLOW',
    'Whooping cough': 'YELLOW',
    'Scombroid food poisoning': 'YELLOW',
    'Anemia': 'YELLOW', 
    'Cluster headache': 'YELLOW', 

    # --- GREEN FLAG (MINOR / HOME CARE) ---
    'Acute laryngitis': 'GREEN',
    'Acute otitis media': 'GREEN', # Ear infection
    'Acute rhinosinusitis': 'GREEN', # Sinus infection
    'Allergic sinusitis': 'GREEN',
    'Bronchitis': 'GREEN',
    'Chronic rhinosinusitis': 'GREEN',
    'GERD': 'GREEN', # Acid reflux
    'Influenza': 'GREEN',
    'Localized edema': 'GREEN',
    'Panic attack': 'GREEN',
    'URTI': 'GREEN', # Common cold
    'Viral pharyngitis': 'GREEN' # Sore throat
}

print(f"Loaded {len(TRIAGE_SEVERITY)} disease protocols into the Safety Net.")

Loaded 49 disease protocols into the Safety Net.


In [4]:
# Cell 2: The Safety Override Logic

def evaluate_clinical_severity(top_3_diseases):
    """
    Scans the Top 3 predictions from PyTorch. 
    Escalates to the highest severity found to prevent missing a critical emergency.
    """
    # Fetch the severity for each disease, defaulting to YELLOW if something is missing
    severities = [TRIAGE_SEVERITY.get(disease, "YELLOW") for disease in top_3_diseases]
    
    # 1. THE RED OVERRIDE
    if "RED" in severities:
        final_severity = "RED"
        system_alert = "[SYSTEM ALERT: CRITICAL EMERGENCY. Instruct the patient to go to the nearest Emergency Room or call an ambulance immediately. DO NOT suggest home remedies. Keep the tone urgent but calm.]"
        
    # 2. THE YELLOW OVERRIDE
    elif "YELLOW" in severities:
        final_severity = "YELLOW"
        system_alert = "[SYSTEM ALERT: URGENT CONDITION. Instruct the patient to schedule an appointment with a doctor soon. Provide safe, temporary symptom management but emphasize medical evaluation.]"
        
    # 3. THE GREEN ALL-CLEAR
    else:
        final_severity = "GREEN"
        system_alert = "[SYSTEM ALERT: MINOR CONDITION. Reassure the patient. Provide safe, over-the-counter home care management and lifestyle adjustments.]"
        
    return final_severity, system_alert

# --- TEST THE OVERRIDE ---
# Imagine PyTorch output this list (GERD is #1, but a Heart Attack is #2)
dummy_predictions = ['GERD', 'Possible NSTEMI / STEMI', 'Panic attack']
severity, alert = evaluate_clinical_severity(dummy_predictions)

print(f"PyTorch Predictions: {dummy_predictions}")
print(f"Calculated Severity: {severity}")
print(f"Gemini Injection: {alert}")

PyTorch Predictions: ['GERD', 'Possible NSTEMI / STEMI', 'Panic attack']
Calculated Severity: RED
Gemini Injection: [SYSTEM ALERT: CRITICAL EMERGENCY. Instruct the patient to go to the nearest Emergency Room or call an ambulance immediately. DO NOT suggest home remedies. Keep the tone urgent but calm.]


In [6]:
# Cell 3: The Chief Medical Officer (Gemini Integration)
import google.generativeai as genai
import os
from dotenv import load_dotenv

# 1. Load the environment variables from your .env file
load_dotenv()

# 2. Securely fetch the key
gemini_key = os.getenv("GEMINI_API_KEY")
if not gemini_key:
    print("⚠️ WARNING: GEMINI_API_KEY not found in .env file!")

# 3. Configure the Gemini SDK
genai.configure(api_key=gemini_key)

def generate_cmo_prompt(top_3_diseases, system_alert):
    """Builds the strict system instruction for Gemini"""
    top_disease = top_3_diseases[0]
    
    prompt = f"""You are the Chief Medical Officer, a highly empathetic, professional, and expert clinical AI assistant.
The PyTorch Triage Engine has analyzed the patient's symptoms. 

The top suspected conditions are: {', '.join(top_3_diseases)}.
The primary focus is: {top_disease}.

🚨 CRITICAL SAFETY OVERRIDE:
{system_alert}

YOUR BEHAVIORAL RULES:
1. THE TONE: Speak directly to the patient with deep empathy and a warm, clinical bedside manner. 
2. NO MATH: DO NOT show the patient the raw list of diseases or any percentages. 
3. PLAIN ENGLISH: Explain what {top_disease} is in very simple, non-medical terms.
4. NO PRESCRIPTIONS: NEVER recommend specific drug dosages or brand-name medications. 
5. THE HANDOFF: Strictly follow the CRITICAL SAFETY OVERRIDE. If it says ER, tell them to go to the ER. If it says home care, give them safe lifestyle tips.
6. THE LOOP: End your message by asking the patient: "Do you have any questions about this condition or what to do next?"
"""
    return prompt

# --- INITIALIZE THE GEMINI CHAT ---
# Pass the prompt as a system instruction
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=generate_cmo_prompt(dummy_predictions, alert)
)

# Start a chat session (this automatically handles conversation history!)
cmo_chat = model.start_chat(history=[])

# Generate the initial "Final Report"
print("Generating Final Report...\n")
response = cmo_chat.send_message("Please give me my triage results.")

print("🩺 CMO Final Report:")
print("-" * 50)
print(response.text)
print("-" * 50)

Generating Final Report...

🩺 CMO Final Report:
--------------------------------------------------
Hello there, I understand you're looking for your triage results, and I want you to know we're here to help you navigate this.

Based on the information you've provided, it is **absolutely critical** that you go to the nearest Emergency Room immediately or call an ambulance right away. This is very important to ensure you receive a thorough and timely evaluation from a medical team. Your safety and well-being are our top priority, and getting immediate medical attention is the most important next step.

One of the conditions we've been looking at is something called GERD, or gastroesophageal reflux disease. This is a common condition where stomach acid frequently flows back up into the tube connecting your mouth and stomach, which can irritate the lining and cause symptoms like heartburn or chest discomfort. However, given all your symptoms, it's essential that these are checked out urgen

In [7]:
# Cell 4: Testing the AI Care Companion Memory

print("🗣️ Patient: But you said it's just acid reflux. Why do I need an ambulance?")
print("-" * 50)

# We send a follow-up question to the SAME chat object
follow_up_response = cmo_chat.send_message("But you said it's just acid reflux. Why do I need an ambulance?")

print("🩺 CMO Response:")
print(follow_up_response.text)

🗣️ Patient: But you said it's just acid reflux. Why do I need an ambulance?
--------------------------------------------------
🩺 CMO Response:
I understand why you might feel that way, and I appreciate you asking for clarification. You're right that we mentioned GERD, which is often called acid reflux. However, it's very important to understand that sometimes symptoms that feel like acid reflux can actually be related to other, more serious conditions that need immediate medical attention.

The symptoms you've described are complex enough that we absolutely cannot take any chances. While GERD is certainly a possibility, there are other potential conditions that share similar symptoms, and some of those require urgent care to ensure your safety.

Because we cannot physically examine you or run tests, the only safe and responsible recommendation is for you to be evaluated by doctors in an Emergency Room right away. They have the tools and expertise to quickly determine exactly what's goi

In [8]:
# Cell 5: Testing the Green Path (Minor Condition)

# 1. Provide safe, green-tier predictions
green_predictions = ['Viral pharyngitis', 'URTI', 'Acute laryngitis']
severity, alert = evaluate_clinical_severity(green_predictions)

print(f"Calculated Severity: {severity}\n")

# 2. Initialize a BRAND NEW chat (so it forgets the heart attack)
green_model = genai.GenerativeModel(
    model_name="gemini-2.5-flash", # or gemini-2.0-flash
    system_instruction=generate_cmo_prompt(green_predictions, alert)
)
green_chat = green_model.start_chat(history=[])

# 3. Get the report
print("Generating Green Path Report...\n")
green_response = green_chat.send_message("Please give me my triage results.")
print("🩺 CMO Final Report (Green):")
print("-" * 50)
print(green_response.text)
print("-" * 50)

# 4. Ask a home-care question
print("\n🗣️ Patient: What should I eat or drink to feel better?")
green_follow_up = green_chat.send_message("What should I eat or drink to feel better?")
print("-" * 50)
print("🩺 CMO Response:")
print(green_follow_up.text)

Calculated Severity: GREEN

Generating Green Path Report...

🩺 CMO Final Report (Green):
--------------------------------------------------
Hello there. I understand you're looking for an update on your symptoms, and I want you to know that we've carefully reviewed everything.

Based on the information we have, it sounds like you're likely experiencing something very common: a **viral sore throat**, also known as viral pharyngitis. This is often part of a common cold or a similar viral infection, and it usually means that a virus is causing inflammation and discomfort in the back of your throat. It's really quite common, especially this time of year, and many people experience it.

The good news is that these types of viral infections typically resolve on their own with a bit of time and self-care at home. You can often feel much better by focusing on comfort and supporting your body's natural healing process.

Here's what we recommend you do at home to help yourself feel more comforta